In [139]:
# Import Required Libraries
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker()

In [140]:
# Create the Billing Table
FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)
DimHospital = pd.read_csv("data/DimHospital.csv")
Billing = pd.merge(FactTable, DimHospital, on='dimLocationPK', how='left')

# Apply Transformations
Billing["PaymentRecieved"] = Billing["Patient_Payment"] + Billing["AR"]
Billing["DiagnosisReportRef"] = "DIAG-" + Billing["FactTablePK"].astype(str)
Billing["ServiceReportRef"] = "SERV-" + Billing["FactTablePK"].astype(str)

column_mapping = {
    "FactTablePK" : "BillNbr",
    "dimDatePostPK" : "BillDate",
    "dimPatientPK" : "PatientNbr",
    "dimPhysicianPK" : "ProviderNbr",
    "LocationName" : "Facility",
    "Gross Expenses" : "BillAmount",
    "Insurance_Payment" : "InsuranceCoverage"
}

# Rename Columns
Billing = Billing.rename(columns=column_mapping)

# Select Columns
Billing = Billing[["BillNbr", "BillDate", "PatientNbr", "ProviderNbr", "Facility", "DiagnosisReportRef", "ServiceReportRef", "BillAmount", "InsuranceCoverage", "PaymentRecieved"]]

# Write the new Data to CSV File
Billing.to_csv('Spaghetti/Billing.csv', index=False)

C:\Users\murta\AppData\Local\Temp\ipykernel_2256\536781932.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  FactTable = pd.read_csv("data/FactTable.csv", parse_dates=['dimDateServicePK'], dayfirst=True)


In [141]:
# Create the Patient Table
DimPatient = pd.read_csv("data/DimPatient.csv")
MinServiceDate = FactTable.groupby("dimPatientPK").agg({"dimDateServicePK": "min"}).reset_index()
Patient = pd.merge(DimPatient, MinServiceDate, on='dimPatientPK', how='left')

# Apply Transformations
Patient.rename(columns={"Year of Birth": "year", "Month": "month", "Day": "day"}, inplace=True)
Patient["DOB"] = pd.to_datetime(Patient[["year", "month", "day"]])
Patient["RegistrationDate"] = Patient["dimDateServicePK"] - pd.to_timedelta(np.random.randint(1, 10, size=len(Patient)), unit='days')
Patient[["Address1", "Address2"]] = Patient.apply(lambda _: pd.Series([fake.secondary_address(), fake.street_address()]), axis=1)

column_mapping = {
    "dimPatientPK" : "PatientNbr",
    "PatientNumber" : "HealthCardNbr",
    "PatientGender" : "Gender",
    "Zip Codes" : "Zip"
}

# Rename Columns
Patient = Patient.rename(columns=column_mapping)

# Select Columns
Patient = Patient[["PatientNbr", "RegistrationDate", "HealthCardNbr", "FirstName", "LastName", "Email", "Gender", "DOB", "Address1", "Address2", "City", "State", "Zip"]]

# Write the new Data to CSV File
Patient.to_csv('Spaghetti/Patient.csv', index=False)